# 4장 시계열 데이터의 시뮬레이션 (택시)

> A unique python library that extends the python programming language and provides utilities that enhance productivity.
- author: "Chansung Park"
- toc: false
- comments: false
- categories: [simulation, taxi]
- permalink: /chapter4-taxi/
- badges: true
- hide_github_badge: true

In [ ]:
import numpy as np 	
		
def taxi_id_number(num_taxis): 
   arr = np.arange(num_taxis)
   np.random.shuffle(arr)
   for i in range(num_taxis):
       yield arr[i]

In [ ]:
ids = taxi_id_number(10) 
print(next(ids))
print(next(ids))
print(next(ids)) 

8
1
3


In [ ]:
def shift_info():
   start_times_and_freqs = [(0, 8), (8, 30), (16, 15)]
   indices               = np.arange(len(start_times_and_freqs)) 
   while True:
       idx = np.random.choice(indices,p=[0.25,0.5,0.25]) 
       start = start_times_and_freqs[idx]
       yield (start[0], start[0] + 7.5, start[1]) 

In [ ]:
def taxi_process(taxi_id_generator, shift_info_generator):
   taxi_id = next(taxi_id_generator)
   shift_start, shift_end, shift_mean_trips = next(shift_info_generator)
   actual_trips = round(np.random.normal(loc   = shift_mean_trips,
                                         scale = 2))
   average_trip_time = 6.5 / shift_mean_trips * 60
   # convert mean trip time to minutes
   # 평균 운행 시간을 분 단위로 변환합니다
   between_events_time = 1.0 / (shift_mean_trips - 1) * 60
   # this is an efficient city where cabs are seldom unused
   # 이 도시는 매우 효율적이어서 모든 택시가 거의 항상 사용됩니다
   time = shift_start
   yield TimePoint(taxi_id, 'start shift', time)
   deltaT = np.random.poisson(between_events_time) / 60
   time += deltaT
   for i in range(actual_trips):
       yield TimePoint(taxi_id, 'pick up ', time)
       deltaT = np.random.poisson(average_trip_time) / 60
       time += deltaT
       yield TimePoint(taxi_id, 'drop off ', time)
       deltaT = np.random.poisson(between_events_time) / 60
       time += deltaT
   deltaT = np.random.poisson(between_events_time) / 60
   time += deltaT
   yield TimePoint(taxi_id, 'end shift ', time) 


In [ ]:
## python 							
from dataclasses import dataclass
 							
@dataclass
class TimePoint:
   taxi_id: int
   name: str		
   time: float
				
   def __lt__(self, other):			
       return self.time < other.time 	

In [ ]:
import queue
			
class Simulator:
   def __init__(self, num_taxis):
       self._time_points = queue.PriorityQueue()
       taxi_id_generator = taxi_id_number(num_taxis)
       shift_info_generator = shift_info()
       self._taxis = [taxi_process(taxi_id_generator,
                                   shift_info_generator) for
                                            i in range(num_taxis)]
       self._prepare_run()

   def _prepare_run(self): 
       for t in self._taxis:
           while True: 
               try:
                   e = next(t)
                   self._time_points.put(e)
               except:
                   break
					
   def run(self): 
       sim_time = 0
       while sim_time < 24:
           if self._time_points.empty():
               break
           p = self._time_points.get()
           sim_time = p.time
           print(p) 	


In [ ]:
sim = Simulator(1000) 
sim.run() 

Streaming output truncated to the last 5000 lines.
TimePoint(taxi_id=799, name='pick up ', time=18.78333333333333)
TimePoint(taxi_id=657, name='pick up ', time=18.78333333333333)
TimePoint(taxi_id=392, name='pick up ', time=18.783333333333335)
TimePoint(taxi_id=683, name='pick up ', time=18.783333333333335)
TimePoint(taxi_id=349, name='drop off ', time=18.783333333333335)
TimePoint(taxi_id=844, name='drop off ', time=18.783333333333335)
TimePoint(taxi_id=711, name='drop off ', time=18.783333333333335)
TimePoint(taxi_id=853, name='drop off ', time=18.783333333333335)
TimePoint(taxi_id=902, name='drop off ', time=18.783333333333335)
TimePoint(taxi_id=844, name='pick up ', time=18.783333333333335)
TimePoint(taxi_id=94, name='drop off ', time=18.783333333333335)
TimePoint(taxi_id=878, name='pick up ', time=18.78333333333334)
TimePoint(taxi_id=67, name='pick up ', time=18.78333333333334)
TimePoint(taxi_id=930, name='drop off ', time=18.78333333333334)
TimePoint(taxi_id=509, name='pick up ',